# 2D Multi-element Airfoil

In [3]:
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import integrate
import os
import csv
%matplotlib inline

In [4]:
# load geometry for each
x_foil, y_foil = np.loadtxt('../AeroPython/lessons/resources/NACA23012_MainFoil.csv', usecols=(0,), unpack=True), np.loadtxt('../AeroPython/lessons/resources/NACA23012_MainFoil.csv', usecols=(1,), unpack=True)
x_flap, y_flap = np.loadtxt('../AeroPython/lessons/resources/NACA23012_FlapFoil.csv', usecols=(0,), unpack=True), np.loadtxt('../AeroPython/lessons/resources/NACA23012_FlapFoil.csv', usecols=(1,), unpack=True)

# plot geometry

ValueError: could not convert string to float: b'1.000000000000000000e+00,0.000000000000000000e+00'

In [5]:
# create panel class
class Panel:
    """
    Contains information related to a panel.
    """
    def __init__(self, xa, ya, xb, yb):
        """
        Initializes the panel.
        
        Sets the end-points and calculates the center-point, length,
        and angle (with the x-axis) of the panel.
        Defines if the panel is located on the upper or lower surface of the geometry.
        Initializes the source-strength, tangential velocity, and pressure coefficient
        of the panel to zero.
        
        Parameters
        ----------
        xa: float
            x-coordinate of the first end-point
        ya: float
            y-coordinate of the first end-point
        xb: float
            x-coordinate of the second end-point
        yb: float
            y-coordinate of the second end-point
        """
        self.xa, self.ya = xa, ya     # panel starting point
        self.xb, self.yb = xb, yb     # panel end point
        
        self.xc, self.yc = (xa+xb)/2, (ya+yb)/2       # panel center
        self.length = math.sqrt((xb-xa)**2+(yb-ya)**2)  # panel length
        
        # orientation of the panel
        if xb-xa <= 0.0:
            self.beta = np.arccos((yb-ya)/self.length)
        elif xb-xa > 0.0:
            self.beta = math.pi + np.arccos(-(yb-ya)/self.length)
            
        # panel location
        if self.beta <= np.pi:
            self.loc = 'upper'    # upper surface
        else:
            self.loc = 'lower'    # lower surface
        
        self.sigma = 0.0     # source strength
        self.vt = 0.0        # tangential velocity
        self.cp = 0.0        # coefficient of pressure

In [13]:
# define panels for main foil
def define_panels_foil(x, y, N=50):
    """
    Discretizes the geometry into panels using the 'cosine' method.
    
    Parameters
    ----------
    x: 1D numpy array of floats
        x-coordinate of the points defining the geometry.
    y: 1D array of floats
        y-coordinate of the points defining the geometry.
    N: integer, optional
        Number of panels;
        default: 50
        
    Returns
    -------
    panels: 1D numpy array of Panel objects
        The list of panels.
    """
    R = (x.max()-x.min())/2                            # radius of the circle
    x_center = (x.max()+x.min())/2                     # x-coordinate of the circle center
    
    theta = np.linspace(0, 2*np.pi, N+1)             # array of angles
    x_circle = x_center + R*np.cos(theta)              # x-coordinates of the circle
    
    x_ends = np.loadtxt('../AeroPython/lessons/resources/MainFoiil_N=50.csv', usecols=(0,), unpack=True)                          # x-coordinate of the panels end-points
    y_ends = np.loadtxt('../AeroPython/lessons/resources/MainFoiil_N=50.csv', usecols=(1,), unpack=True)                     # y-coordinate of the panels end-points
    
    # create panels
    panels = np.empty(N, dtype = object)
    for i in range(N):
        panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
        
    return panels

In [14]:
# define panels for flap
def define_panels_flap(x, y, N=50):
    """
    Discretizes the geometry into panels using the 'cosine' method.
    
    Parameters
    ----------
    x: 1D numpy array of floats
        x-coordinate of the points defining the geometry.
    y: 1D array of floats
        y-coordinate of the points defining the geometry.
    N: integer, optional
        Number of panels;
        default: 50
        
    Returns
    -------
    panels: 1D numpy array of Panel objects
        The list of panels.
    """
    R = (x.max()-x.min())/2                            # radius of the circle
    x_center = (x.max()+x.min())/2                     # x-coordinate of the circle center
    
    theta = np.linspace(0, 2*np.pi, N+1)             # array of angles
    x_circle = x_center + R*np.cos(theta)              # x-coordinates of the circle
    
    x_ends = np.loadtxt('../AeroPython/lessons/resources/FlapFoiil_N=50.csv', usecols=(0,), unpack=True)                          # x-coordinate of the panels end-points
    y_ends = np.loadtxt('../AeroPython/lessons/resources/FlapFoiil_N=50.csv', usecols=(1,), unpack=True)                     # y-coordinate of the panels end-points
    
    # create panels
    panels = np.empty(N, dtype = object)
    for i in range(N):
        panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
        
    return panels

In [15]:
# discretize geometry into panels
panels_foil = define_panels_foil(x_foil, y_foil, N=50)
panels_flap = define_panels_flap(x_flap, y_flap, N=50)

NameError: name 'x_foil' is not defined

In [17]:
# define integral for the source and vortex contributions (can use for both main foil and flap)
def integral(x, y, panel, dxdk, dydk):
    """
    Evaluates the contribution from a panel at a given point.
    
    Parameters
    ----------
    x: float
        x-coordinate of the target point.
    y: float
        y-coordinate of the target point.
    panel: Panel object
        Panel whose contribution is evaluated.
    dxdk: float
        Value of the derivative of x in a certain direction.
    dydk: float
        Value of the derivative of y in a certain direction.
    
    Returns
    -------
    Contribution from the panel at a given point (x, y).
    """
    def integrand(s):
        return ( ((x - (panel.xa - numpy.sin(panel.beta)*s))*dxdk
                  +(y - (panel.ya + numpy.cos(panel.beta)*s))*dydk)
                / ((x - (panel.xa - numpy.sin(panel.beta)*s))**2
                   +(y - (panel.ya + numpy.cos(panel.beta)*s))**2) )
    return integrate.quad(integrand, 0.0, panel.length)[0]

In [22]:
# source contribution for main foil
def source_contribution_normal_mainfoil(panels_foil):
    """
    Builds the source contribution matrix for the normal velocity.
    
    Parameters
    ----------
    panels: 1D array of Panel objects
        List of panels.
    
    Returns
    -------
    A: 2D Numpy array of floats
        Source contribution matrix.
    """
    A_foil = numpy.empty((panels.size, panels.size), dtype=float)
    # source contribution on a panel from itself
    numpy.fill_diagonal(A_foil, 0.5)
    # source contribution on a panel from others
    for i, panel_i in enumerate(panels):
        for j, panel_j in enumerate(panels):
            if i != j:
                A_foil[i, j] = 0.5/numpy.pi*integral(panel_i.xc, panel_i.yc, 
                                                panel_j,
                                                numpy.cos(panel_i.beta),
                                                numpy.sin(panel_i.beta))
    return A_foil

In [23]:
# source contribution for flap
def source_contribution_normal_flap(panels_flap):
    """
    Builds the source contribution matrix for the normal velocity.
    
    Parameters
    ----------
    panels: 1D array of Panel objects
        List of panels.
    
    Returns
    -------
    A: 2D Numpy array of floats
        Source contribution matrix.
    """
    A_flap = numpy.empty((panels.size, panels.size), dtype=float)
    # source contribution on a panel from itself
    numpy.fill_diagonal(A_flap, 0.5)
    # source contribution on a panel from others
    for i, panel_i in enumerate(panels):
        for j, panel_j in enumerate(panels):
            if i != j:
                A_flap[i, j] = 0.5/numpy.pi*integral(panel_i.xc, panel_i.yc, 
                                                panel_j,
                                                numpy.cos(panel_i.beta),
                                                numpy.sin(panel_i.beta))
    return A_flap

In [24]:
# define vortex contribution for main foil
def vortex_contribution_normal_mainfoil(panels_foil):
    """
    Builds the vortex contribution matrix for the normal velocity.
    
    Parameters
    ----------
    panels: 1D array of Panel objects
        List of panels.
    
    Returns
    -------
    A: 2D Numpy array of floats
        Vortex contribution matrix.
    """
    A_foil = numpy.empty((panels.size, panels.size), dtype=float)
    # vortex contribution on a panel from itself
    numpy.fill_diagonal(A_foil, 0.0)
    # vortex contribution on a panel from others
    for i, panel_i in enumerate(panels):
        for j, panel_j in enumerate(panels):
            if i != j:
                A_foil[i, j] = -0.5/numpy.pi*integral(panel_i.xc, panel_i.yc, 
                                                 panel_j,
                                                 numpy.sin(panel_i.beta),
                                                 -numpy.cos(panel_i.beta))
    return A_foil

In [25]:
# define vortex contribution for flap
def vortex_contribution_normal_flap(panels_flap):
    """
    Builds the vortex contribution matrix for the normal velocity.
    
    Parameters
    ----------
    panels: 1D array of Panel objects
        List of panels.
    
    Returns
    -------
    A: 2D Numpy array of floats
        Vortex contribution matrix.
    """
    A_flap = numpy.empty((panels.size, panels.size), dtype=float)
    # vortex contribution on a panel from itself
    numpy.fill_diagonal(A_flap, 0.0)
    # vortex contribution on a panel from others
    for i, panel_i in enumerate(panels):
        for j, panel_j in enumerate(panels):
            if i != j:
                A_flap[i, j] = -0.5/numpy.pi*integral(panel_i.xc, panel_i.yc, 
                                                 panel_j,
                                                 numpy.sin(panel_i.beta),
                                                 -numpy.cos(panel_i.beta))
    return A_flap

In [26]:
# create source and vortex matrices
A_source = source_contribution_normal_mainfoil(panels_foil) + source_contribution_normal_flap(panels_flap)
B_vortex = vortex_contribution_normal_mainfoil(panels_foil) + vortex_contribution_normal_flap(panels_flap)

NameError: name 'panels_foil' is not defined

In [27]:
def kutta_condition_mainfoil(A_source, B_vortex):
    """
    Builds the Kutta condition array.
    
    Parameters
    ----------
    A_source: 2D Numpy array of floats
        Source contribution matrix for the normal velocity.
    B_vortex: 2D Numpy array of floats
        Vortex contribution matrix for the normal velocity.
    
    Returns
    -------
    b: 1D Numpy array of floats
        The left-hand side of the Kutta-condition equation.
    """
    b = numpy.empty(A_source.shape[0]+1, dtype=float)
    # matrix of source contribution on tangential velocity
    # is the same than
    # matrix of vortex contribution on normal velocity
    b[:-1] = B_vortex[0, :] + B_vortex[-1, :]
    # matrix of vortex contribution on tangential velocity
    # is the opposite of
    # matrix of source contribution on normal velocity
    b[-1] = - numpy.sum(A_source[0, :] + A_source[-1, :])
    return b